In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

################
# PARSE FUNCS  #
################
def get_html(url):
    response = requests.get(url)
    return response.text


def get_common_table(page):
    soap = BeautifulSoup(page, "lxml")
    return soap.find('table', id='t_common')


def parse_headers(table):
    headers = table.find_all('tr')[1].find_all('td')
    return [header.text for header in headers]


def parse_rows(table):
    rows = table.find_all('tr')[2:]
    data = []
    for r in rows:
        rdata = []
        for td in r.find_all('td'):
            rdata.append(td.text.strip())
        data.append(rdata)
    return data


################
# PANDAS FUNCS #
################
def page_to_df(url):
    common_table = get_common_table(get_html(url))
    table, headers = parse_rows(common_table), parse_headers(common_table)
    df = pd.DataFrame(table, columns=headers)
    return df


def non_outsiders(df):
    return df[df['Примечание'] == '']


def only_originals(df):
    return df[df['Оригинал документа об образовании'] == 'да']


def add_institution_column(df, institution_name):
    df['Направление'] = institution_name
    return df.reset_index(drop=True)


def delete_user(df, id):
    user_index = df[df['id абитуриента/СНИЛС'] == id].index
    df = df.drop(user_index)
    return df


def get_passing_users_by_pr(df = None, pr = 1, count = 10):
    df = df.head(count)
    return df[df['Приоритет'] == f'{pr}']

In [19]:
def to_admite(directions = [], pr = 1):
    if len(directions) == 0:
        return ValueError('Нет напрвлений')
    if pr <= 0:
        return ValueError('Неправильный приоритет')
        
    user_list = []
    
    for d in directions:
        # берем верхушку с приоритетом pr
        free_places = d.places
        top_stdt = d.df.head(free_places)
        top_stdt_by_pr = top_stdt[top_stdt['Приоритет'] == f'{pr}']

        # зачисляем
        d.admitted = pd.concat([d.admitted, top_stdt_by_pr])
        d.places -= top_stdt_by_pr.shape[0]
        # print(f'на {d.name} осталось: {d.places}, зачислили {top_stdt_by_pr.shape[0]}')

        # запоминаем кто поступил
        user_list += top_stdt_by_pr['id абитуриента/СНИЛС'].to_list()

    for d in directions:
        d.df = d.df[~d.df['id абитуриента/СНИЛС'].isin(user_list)]
    
    return user_list    

In [20]:
from settings import Direction, Tables

cfg = Tables()

# заполнение датафреймов
for i in cfg.get_directions():
    df = add_institution_column(non_outsiders(page_to_df(i.url)), f'{i.name}')
    i.df = df

stdt = pd.concat(cfg.get_dfs())
unique_stdt = stdt['id абитуриента/СНИЛС'].nunique()
print(f'Количество уникальных студентов В КОНКУРСЕ: {unique_stdt}')

# фильтр по оригам
for i in cfg.get_directions():
    df = only_originals(i.df)
    i.df = df

stdt = pd.concat(cfg.get_dfs())
unique_stdt = stdt['id абитуриента/СНИЛС'].nunique()
print(f'Количество уникальных студентов С ОРИГИНАЛАМИ: {unique_stdt}')

Количество уникальных студентов В КОНКУРСЕ: 262
Количество уникальных студентов С ОРИГИНАЛАМИ: 134


In [21]:
adm = 0
for p in range(1, 15):
    wave = to_admite(cfg.get_directions(), pr=p)
    # print(f'для приритета {p} зачислено {len(wave)}\n')
    adm += len(wave)

free = 0
for d in cfg.get_directions():
    free += d.places

adm, free

(116, 20)

In [12]:
final_fiit = cfg.IB.admitted.sort_values('Сумма конкурсных баллов')
final_fiit['Сумма конкурсных баллов'] = pd.to_numeric(final_fiit['Сумма конкурсных баллов'], errors='coerce')
final_fiit = final_fiit.sort_values('Сумма конкурсных баллов', ascending=False)
final_fiit = final_fiit.reset_index(drop=True)
final_fiit

,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
0,1,163-687-560-07,91,нет,1,да,участвует в конкурсе,,IB
1,3,154-750-695-84,89,нет,1,да,участвует в конкурсе,,IB
2,4,160-033-231-97,89,нет,1,да,участвует в конкурсе,,IB
3,5,157-743-184-94,88,нет,1,да,участвует в конкурсе,,IB
4,6,142-568-974-91,87,нет,2,да,участвует в конкурсе,,IB
5,7,160-690-769-79,86,нет,1,да,участвует в конкурсе,,IB
6,15,145-778-121-92,80,нет,2,да,участвует в конкурсе,,IB
7,18,160-690-635-66,74,нет,4,да,участвует в конкурсе,,IB


In [13]:
import pandas as pd

INN = '145-674-739-98'

def get_pos():
    res = pd.DataFrame()
    for df in cfg.get_admitted():
        filtered_df = df[df['id абитуриента/СНИЛС'] == INN]
        if not filtered_df.empty:
            res = filtered_df
            break
    return res if not res.empty else pd.DataFrame(columns=df.columns)

result = get_pos()
result


,№,id абитуриента/СНИЛС,Сумма конкурсных баллов,Преимущественное право,Приоритет,Оригинал документа об образовании,Статус,Примечание,Направление
30,31,145-674-739-98,75,нет,3,да,участвует в конкурсе,,PM
